# 1. Import

In [4]:
import h5py
import torch
import numpy as np
import collections


# 2. Add feature

In [ ]:
def write_single_graph(graph_id, x, edge_index, y, edge_y):
    '''
    store into hdf5 file
    '''
    with h5py.File("/content/drive/Shareddrives/botnetdata/p2p/processed/p2p_val.hdf5", 'a') as f:
      f.create_dataset(f'{graph_id}/x', data=x, dtype = 'float32')
      f.create_dataset(f'{graph_id}/edge_index', data=edge_index, dtype = 'int64')
      f.create_dataset(f'{graph_id}/y', data=y, dtype = 'uint8')
      f.create_dataset(f'{graph_id}/edge_y', data=y, dtype = 'uint8')

In [ ]:
num_train_graph = 768
num_val_graph = 96
num_test_graph = 96
with h5py.File("/content/drive/Shareddrives/botnetdata/P2P/processed/p2p_val.hdf5", 'r') as f:

  for graph_id in range(num_val_graph):
    print(f[f'{graph_id}'])
    num_node = f[f'{graph_id}']['x'].shape[0]
    edge_index = f[f'{graph_id}']['edge_index']
    label = f[f'{graph_id}']['y']
    edge_y = f[f'{graph_id}']['edge_y']

    node_degree = torch.zeros(f[f'{graph_id}']['x'].shape)

    source_node, destination_node = edge_index
    concat = np.concatenate((source_node, destination_node), axis=0)
    collection = collections.Counter(concat.flatten())

    for key in collection:
      node_degree[key] = collection[key]

    write_single_graph(graph_id=graph_id, x=node_degree, edge_index=edge_index, y=label, edge_y=edge_y)  

# 3. Normalization

In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import h5py
import torch
import numpy as np
import collections


In [6]:
num_train_graph = 768
num_val_graph = 96
num_test_graph = 96
arr_concat = None
with h5py.File("/content/drive/Shareddrives/botnetdata/p2p/processed/p2p_train.hdf5", 'r') as f:
  for graph_id in range(num_train_graph):
    print(f[f'{graph_id}'])
    num_nodes = f[f'{graph_id}']['x'].shape[0]

    if graph_id == 0:
      arr_concat = f[f'{graph_id}']['x'][:]
    else:
      arr_concat = np.vstack((arr_concat, f[f'{graph_id}']['x'][:]))
    
    print(f[f'{graph_id}']['x'][:])
    print(arr_concat)


Streaming output truncated to the last 5000 lines.
 [  8.]
 [  6.]
 ...
 [  6.]
 [  6.]
 [  6.]]
<HDF5 group "/432" (4 members)>
[[ 10.]
 [128.]
 [624.]
 ...
 [  6.]
 [  8.]
 [  8.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  6.]
 [  8.]
 [  8.]]
<HDF5 group "/433" (4 members)>
[[268.]
 [896.]
 [690.]
 ...
 [  6.]
 [  6.]
 [  6.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  6.]
 [  6.]
 [  6.]]
<HDF5 group "/434" (4 members)>
[[292.]
 [ 12.]
 [ 16.]
 ...
 [  6.]
 [  6.]
 [  6.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  6.]
 [  6.]
 [  6.]]
<HDF5 group "/435" (4 members)>
[[ 18.]
 [ 10.]
 [524.]
 ...
 [  8.]
 [  6.]
 [  6.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  8.]
 [  6.]
 [  6.]]
<HDF5 group "/436" (4 members)>
[[1206.]
 [  80.]
 [   8.]
 ...
 [   6.]
 [   6.]
 [   6.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  6.]
 [  6.]
 [  6.]]
<HDF5 group "/437" (4 members)>
[[ 6.]
 [28.]
 [60.]
 ...
 [ 6.]
 [ 6.]
 [ 6.]]
[[628.]
 [  8.]
 [  6.]
 ...
 [  6.]
 [  6.]
 [  6.]]
<HDF5 group "/438" (4 members)>
[[ 32.]
 [ 66.]
 [672.]
 ...
 [

In [7]:
def write_single_graph_after_normalization(graph_id, x, edge_index, y, edge_y):
    '''
    store into hdf5 file
    '''
    with h5py.File("/content/drive/Shareddrives/botnetdata/p2p_normalization/processed/p2p_test.hdf5", 'a') as f:
      f.create_dataset(f'{graph_id}/x', data=x, dtype = 'float32')
      f.create_dataset(f'{graph_id}/edge_index', data=edge_index, dtype = 'int64')
      f.create_dataset(f'{graph_id}/y', data=y, dtype = 'uint8')
      f.create_dataset(f'{graph_id}/edge_y', data=y, dtype = 'uint8')

In [ ]:
scaler = MinMaxScaler()
scaler.fit(arr_concat)
with h5py.File("/content/drive/Shareddrives/botnetdata/p2p/processed/p2p_test.hdf5", 'r') as fil:
  for graph_id in range(num_test_graph):
    print(fil[f'{graph_id}'])
    num_node = fil[f'{graph_id}']['x'].shape[0]
    edge_index = fil[f'{graph_id}']['edge_index']
    label = fil[f'{graph_id}']['y']
    edge_y = fil[f'{graph_id}']['edge_y']
    x =  fil[f'{graph_id}']['x'][:]
    print(x.shape)
    x_tran = scaler.transform(x)

    write_single_graph_after_normalization(graph_id=graph_id, x=x_tran, edge_index=edge_index, y=label, edge_y=edge_y)